In [ ]:
# Will be three modeling steps:
# Model inputs: State budget, max travel distance
# 1. Demand maximation (s/t cost, distance), gives # people we can route to a hub
# 2. Aggregate distance minimization
# 3. Use (x) * # people, (y) * distance for cost minimization

In [8]:
import os
from datetime import datetime
import pandas as pd
import numpy as np
from pyomo.environ import *
import pyomo.opt as pyopt

import geopandas as gpd
import pandas as pd
import numpy as np
from statistics import mean

import itertools

import matplotlib.pyplot as plt
import plotly.express as px
import shapely
import folium
import plotly.graph_objects as go # or plotly.express as px

from data_cleaning_cmm import (blockgroup_pop_dict, bg_ces_dict,
                               dist_to_site_contra_costa_df, dist_to_site_contra_costa_dict,
                               dist_to_site_contra_costa_walk_df, dist_to_site_contra_costa_walk_dict,
                               dist_to_site_wilmington_df, dist_to_site_wilmington_dict,
                               dist_to_site_wilmington_walk_df, dist_to_site_wilmington_walk_dict,
                           county_prop_ealp_dict, site_kw_occ_dict,
                           site_sqft_dict, site_cost_dict)

In [9]:
# Base model
import deployment_models
import importlib
importlib.reload(deployment_models)


<module 'deployment_models' from '/Users/clairemorton/Documents/__PSE/sgc-deployment-scenarios/deployment_models.py'>

In [10]:
# Demand maximization s/t budget
######### Basic demand maximization #############
def model_demand_max(max_cost, prop_served_scale_factor, model_dict, model_base):
    # Demand maximization objective and constraint for total cost and to meet defined proportion of total population
    model_key = "demand_max"
    model_dict[model_key] = model_base.clone()
    model_dict[model_key] = deployment_models.constrain_total_cost(model_dict[model_key],max_cost)
    model_dict[model_key] = deployment_models.add_demand_maximization_objective(model_dict[model_key])
    results = SolverFactory('gurobi').solve(model_dict[model_key])
    #print(results)
    var_hub_yn, var_bg_pop, var_prop_served, var_distance_matrix = deployment_models.get_variables_from_model(model_dict[model_key])

    # Calculate number of people served
    num_served_bg = var_prop_served.sum(axis = 1)*var_bg_pop['POPULATION']
    prop_served = sum(num_served_bg.loc[num_served_bg != 0])/sum(var_bg_pop['POPULATION'])
    scaled_prop_served = prop_served * prop_served_scale_factor
    return scaled_prop_served

In [11]:
######### Basic p median #############
def model_p_median(max_cost, scaled_prop_served, agg_dist_scale_factor, model_dict, model_base):
    # p median objective and constraint for total cost and to meet defined proportion of total population
    model_key = "p_median"
    model_dict[model_key] = model_base.clone()
    model_dict[model_key] = deployment_models.constrain_total_cost(model_dict[model_key],max_cost)
    model_dict[model_key] = deployment_models.constrain_min_total_pop(model_dict[model_key],scaled_prop_served)
    model_dict[model_key] = deployment_models.add_p_median_objective(model_dict[model_key])
    results = SolverFactory('gurobi').solve(model_dict[model_key])
    #print(results)
    var_hub_yn, var_bg_pop, var_prop_served, var_distance_matrix = deployment_models.get_variables_from_model(model_dict[model_key])

    # Create set of blockgroup, site pairs within five miles of each other
    def filter_to_nearby_sites(bg, site):
        return not np.isnan(var_distance_matrix.loc[bg, site])

    bg_site_pairs = list(itertools.product(var_distance_matrix.columns,var_distance_matrix.index))
    bg_site_pairs = [x for x in bg_site_pairs if filter_to_nearby_sites(x[1], x[0])]

    # Calculate aggregate distance and scaled aggregated distance
    agg_dist = sum([var_bg_pop.loc[bg, 'POPULATION'] * var_distance_matrix.loc[bg, site] * var_prop_served.loc[bg, site] for site, bg in bg_site_pairs])
    scaled_agg_dist = agg_dist * agg_dist_scale_factor
    return scaled_agg_dist

In [84]:
######### Cost Minimization #############
# cost minimization objective and constraints for aggregate distance and proportion of the population served
# returns dataframe of whether or not hubs were built and objective value (cost to build)
def model_min_cost(scaled_agg_dist, scaled_prop_served, model_dict, model_base):
    model_key = "cost"
    model_dict[model_key] = model_base.clone()
    model_dict[model_key] = deployment_models.constrain_maximum_agg_distance(model_dict[model_key],scaled_agg_dist)
    model_dict[model_key] = deployment_models.constrain_min_total_pop(model_dict[model_key],scaled_prop_served)
    model_dict[model_key] = deployment_models.add_cost_minimzation_objective(model_dict[model_key])
    results = SolverFactory('gurobi').solve(model_dict[model_key])
    #print(results)
    obj_val = model_dict[model_key].obj_cost_minimzation()
    var_hub_yn, var_bg_pop, var_prop_served, var_distance_matrix = deployment_models.get_variables_from_model(model_dict[model_key])
    return var_hub_yn, obj_val

# cost minimization objective and constraints for aggregate distance and proportion of the population served
# uses weighted total population constraint based on CES score
# returns dataframe of whether or not hubs were built and objective value (cost to build)
def model_min_cost_ces(scaled_agg_dist, scaled_prop_served, min_prop_ej, ej_cutoff, model_dict, model_base):
    model_key = "cost"
    model_dict[model_key] = model_base.clone()
    model_dict[model_key] = deployment_models.constrain_maximum_agg_distance(model_dict[model_key],scaled_agg_dist)
    model_dict[model_key] = deployment_models.prioritize_CES(model_dict[model_key],min_prop_ej,ej_cutoff) # must serve min_prop_ej prop for BG with CES scores greater than ej_cutoff
    model_dict[model_key] = deployment_models.constrain_min_total_pop(model_dict[model_key],scaled_prop_served)
    model_dict[model_key] = deployment_models.add_cost_minimzation_objective(model_dict[model_key])
    results = SolverFactory('gurobi').solve(model_dict[model_key])
    #print(results)
    obj_val = model_dict[model_key].obj_cost_minimzation()
    var_hub_yn, var_bg_pop, var_prop_served, var_distance_matrix = deployment_models.get_variables_from_model(model_dict[model_key])
    return var_hub_yn, obj_val

In [89]:
######### Cost Minimization from demand maximization and p median #############
# takes distance to sites dataframe (constrains geometry of interest), 
#    amount we can spend on hubs, miles we can go to a hub, scale factors
# returns locations of built hubs
def model_func(dist_to_site_df, max_cost, prop_served_scale_factor, agg_dist_scale_factor):
    # Set up base model
    model_base, bg_with_no_hub = deployment_models.build_base_model(site_cost_dict, site_kw_occ_dict, blockgroup_pop_dict, bg_ces_dict, dist_to_site_df)
    model_dict = dict()
    
    prop_served_output = model_demand_max(max_cost, prop_served_scale_factor, model_dict, model_base)
    agg_dist_output = model_p_median(max_cost, prop_served_output, agg_dist_scale_factor, model_dict, model_base)
    return model_min_cost(agg_dist_output, prop_served_output, model_dict, model_base)
    
# takes distance to sites dataframe (constrains geometry of interest), 
#    amount we can spend on hubs, miles we can go to a hub, scale factors
# uses CES constraint -- way this works now is you have to meet 75% of the demand model output (which is scaled via scale factor)
#    and 125% of the scaled demand model output in BGs over CES 80th percentile
# returns locations of built block groups
def model_func_ces(dist_to_site_df, max_cost, prop_served_scale_factor, agg_dist_scale_factor):
    # Set up base model
    model_base, bg_with_no_hub = deployment_models.build_base_model(site_cost_dict, site_kw_occ_dict, blockgroup_pop_dict, bg_ces_dict, dist_to_site_df)
    model_dict = dict()
    
    prop_served_output = model_demand_max(max_cost, prop_served_scale_factor, model_dict, model_base)
    agg_dist_output = model_p_median(max_cost, prop_served_output, agg_dist_scale_factor, model_dict, model_base)
    return model_min_cost_ces(agg_dist_output, prop_served_output*.75, prop_served_output*1.25, .8, model_dict, model_base)
    



In [81]:
# function that takes resilience hub or block group data and id,
# and returns coordinates of hub or block group corresponding to id
def locate(data, id, col):
    row = data.loc[data[col] == id]
    return [row['LAT'].iloc[0], row['LON'].iloc[0]]

# Map hub locations based on built_yn
def map_hubs(built_yn, site_data):
    sites = pd.DataFrame(columns = ['LAT', 'LON'])
    hubs = built_yn.loc[built_yn['BUILT'] == 1].index
    for site in hubs:
        site_pt = locate(site_data, site, 'id_site')
        sites.loc[len(sites.index)] = site_pt
    fig = px.scatter_mapbox(lat=sites['LAT'], lon=sites['LON'], mapbox_style="open-street-map", zoom=10, color_discrete_sequence = ["black"])
    fig.show()

In [68]:
# Load sites data
ca_albers_nad83 = 'NAD_1983_California_Teale_Albers_FtUS'
nad83 = 'EPSG:4629'
wgs84 = 'EPSG:4326'

# Building candidate sites GeoDataFrame
sites_path = os.path.join(os.getcwd(), 'data', 'candidate_site_campuses_2021-11-17', 'candidate_sites_campuses.csv')
sites_df_raw = pd.read_csv(sites_path)
sites_df_raw = sites_df_raw.loc[sites_df_raw['cat_site'] != 'X', ['id_site', 'cat_site', 'SQFT_ROOF', 'LON', 'LAT']]
sites_geom = gpd.points_from_xy(sites_df_raw.LON, sites_df_raw.LAT, crs = nad83)
sites_gdf = gpd.GeoDataFrame(sites_df_raw, geometry = sites_geom, crs = nad83)



In [82]:
# apply cost minimization combined function

max_cost = 5000000 # $ that we can spend on hubs
prop_served_scale_factor = .75
agg_dist_scale_factor = 1.25

var_hub_yn_contra_costa_drive = model_func(dist_to_site_contra_costa_df, max_cost, prop_served_scale_factor, agg_dist_scale_factor)[0]
var_hub_yn_contra_costa_walk = model_func(dist_to_site_contra_costa_walk_df, max_cost, prop_served_scale_factor, agg_dist_scale_factor)[0]

var_hub_yn_wilmington_drive = model_func(dist_to_site_wilmington_df, max_cost, prop_served_scale_factor, agg_dist_scale_factor)[0]
var_hub_yn_wilmington_walk = model_func(dist_to_site_wilmington_walk_df, max_cost, prop_served_scale_factor, agg_dist_scale_factor)[0]



In [90]:
var_hub_yn_contra_costa_drive_ces = model_func_ces(dist_to_site_contra_costa_df, max_cost, prop_served_scale_factor, agg_dist_scale_factor)[0]
var_hub_yn_contra_costa_walk_ces = model_func_ces(dist_to_site_contra_costa_walk_df, max_cost, prop_served_scale_factor, agg_dist_scale_factor)[0]

var_hub_yn_wilmington_drive_ces = model_func_ces(dist_to_site_wilmington_df, max_cost, prop_served_scale_factor, agg_dist_scale_factor)[0]
var_hub_yn_wilmington_walk_ces = model_func_ces(dist_to_site_wilmington_walk_df, max_cost, prop_served_scale_factor, agg_dist_scale_factor)[0]


In [26]:
# Quick look at walking vs driving differences
ind_drive_cc = var_hub_yn_contra_costa_drive.loc[var_hub_yn_contra_costa_drive['BUILT'] == 1].index
ind_walk_cc = var_hub_yn_contra_costa_walk.loc[var_hub_yn_contra_costa_walk['BUILT'] == 1].index
print(set(ind_drive_cc).symmetric_difference(set(ind_walk_cc)))

ind_drive_w = var_hub_yn_wilmington_drive.loc[var_hub_yn_wilmington_drive['BUILT'] == 1].index
ind_walk_w = var_hub_yn_wilmington_walk.loc[var_hub_yn_wilmington_walk['BUILT'] == 1].index
print(set(ind_drive_w).symmetric_difference(set(ind_walk_w)))
#print(len(ind_drive_cc))
print(ind_drive_cc)
print(ind_walk_cc)
print(ind_drive_w)
print(ind_walk_w)

{'129601815', '673519917', '510044766', '517797167', '542094165', '460682836', '509324175', '460682860', '509324172', '509486922'}
set()
Index(['460682836', '509324175', '509486922', '542094165', '362436440',
       '559359901', '230527917', '673519917', '464044699', '479032686',
       '481038146'],
      dtype='object')
Index(['460682860', '509324172', '517797167', '362436440', '559359901',
       '129601815', '230527917', '510044766', '464044699', '479032686',
       '481038146'],
      dtype='object')
Index(['407808844', '796874745'], dtype='object')
Index(['407808844', '796874745'], dtype='object')


In [91]:
# Quick look at walking vs driving differences (CES)
ind_drive_cc = var_hub_yn_contra_costa_drive_ces.loc[var_hub_yn_contra_costa_drive_ces['BUILT'] == 1].index
ind_walk_cc = var_hub_yn_contra_costa_walk_ces.loc[var_hub_yn_contra_costa_walk_ces['BUILT'] == 1].index
print(set(ind_drive_cc).symmetric_difference(set(ind_walk_cc)))

ind_drive_w = var_hub_yn_wilmington_drive_ces.loc[var_hub_yn_wilmington_drive_ces['BUILT'] == 1].index
ind_walk_w = var_hub_yn_wilmington_walk_ces.loc[var_hub_yn_wilmington_walk_ces['BUILT'] == 1].index
print(set(ind_drive_w).symmetric_difference(set(ind_walk_w)))
#print(len(ind_drive_cc))
print(ind_drive_cc)
print(ind_walk_cc)
print(ind_drive_w)
print(ind_walk_w)

{'595180496', '420198791', '510044766', '255496981', '250909848', '294779081', '482067952', '509486922', '205384770', '420207183', '460261835', '479032693', '480364415', '444806750', '232019756', '509324172', '460468494', '460261840', '673519917', '131837314', '481038440', '304485786', '460678854', '230527917', '481599042', '542094165', '326482018', '459843920', '460682860', '305323076', '31279168'}
{'796874756', '407624485', '407808844', '797745383'}
Index(['31583931', '31357750', '420207183', '326482018', '460682836',
       '460678854', '460261844', '459843920', '460682860', '460261835',
       '595180496', '31279168', '516933915', '31330524', '542094165',
       '255496981', '362436440', '552332664', '420198791', '230527917',
       '673519917', '205384770', '41783060', '909193794', '232019756',
       '484701172', '294779081', '464044699', '479032686', '479032693',
       '480364415', '480382407', '481038146', '481038440', '305323076',
       '481599042', '482067952', '250909848',

In [92]:
# Quick look at CES vs non-CES differences
ind_drive_cc_ces = var_hub_yn_contra_costa_drive_ces.loc[var_hub_yn_contra_costa_drive_ces['BUILT'] == 1].index
ind_drive_cc = var_hub_yn_contra_costa_drive.loc[var_hub_yn_contra_costa_drive['BUILT'] == 1].index
print(set(ind_drive_cc_ces).symmetric_difference(set(ind_drive_cc)))

ind_walk_cc_ces = var_hub_yn_contra_costa_walk_ces.loc[var_hub_yn_contra_costa_walk_ces['BUILT'] == 1].index
ind_walk_cc = var_hub_yn_contra_costa_walk.loc[var_hub_yn_contra_costa_walk['BUILT'] == 1].index
print(set(ind_walk_cc_ces).symmetric_difference(set(ind_walk_cc)))

ind_drive_w_ces = var_hub_yn_wilmington_drive_ces.loc[var_hub_yn_wilmington_drive_ces['BUILT'] == 1].index
ind_drive_w = var_hub_yn_wilmington_drive.loc[var_hub_yn_wilmington_drive['BUILT'] == 1].index
print(set(ind_drive_w_ces).symmetric_difference(set(ind_drive_w)))

ind_walk_w_ces = var_hub_yn_wilmington_walk_ces.loc[var_hub_yn_wilmington_walk_ces['BUILT'] == 1].index
ind_walk_w = var_hub_yn_wilmington_walk.loc[var_hub_yn_wilmington_walk['BUILT'] == 1].index
print(set(ind_walk_w_ces).symmetric_difference(set(ind_walk_w)))

print(ind_drive_cc_ces)
print(ind_drive_cc)

print(ind_walk_cc_ces)
print(ind_walk_cc)

print(ind_drive_w_ces)
print(ind_drive_w)

print(ind_walk_w_ces)
print(ind_walk_w)

{'595180496', '420198791', '510044766', '255496981', '31357750', '250909848', '294779081', '41783060', '482067952', '509324175', '205384770', '479032686', '420207183', '460261835', '444806750', '232019756', '552332664', '559359901', '31583931', '481038440', '480382407', '516933915', '477480658', '460678854', '909193794', '481599042', '484701172', '494592360', '326482018', '31330524', '459843920', '460682860', '305323076', '31279168'}
{'484998348', '559359901', '31357750', '294779081', '41783060', '31583931', '460682836', '131837314', '460468494', '480382407', '516933915', '477480658', '460261844', '909193794', '509486922', '230527917', '129601815', '484701172', '494592360', '31330524', '460682860', '552332664', '460261840'}
{'800655210', '407624485', '796874756', '407809112', '407808844', '797207163', '797744802'}
{'800655210', '407809112', '797745383', '797207163', '797744802'}
Index(['31583931', '31357750', '420207183', '326482018', '460682836',
       '460678854', '460261844', '4598

In [93]:
map_hubs(var_hub_yn_contra_costa_walk, sites_gdf)

In [94]:
map_hubs(var_hub_yn_contra_costa_walk_ces, sites_gdf)

In [97]:
map_hubs(var_hub_yn_contra_costa_drive, sites_gdf)

In [96]:
map_hubs(var_hub_yn_contra_costa_drive_ces, sites_gdf)

In [33]:
map_hubs(var_hub_yn_wilmington_walk, sites_gdf)

In [34]:
map_hubs(var_hub_yn_wilmington_walk_ces, sites_gdf)

In [35]:
map_hubs(var_hub_yn_wilmington_drive, sites_gdf)

In [36]:
map_hubs(var_hub_yn_wilmington_drive_ces, sites_gdf)